## Boston Housing Prices Dataset

In this version, we stack RandomForest and Support Vector Regression models

In [1]:
import warnings
import numpy as np 
import pandas as pd 

from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedKFold, KFold, ParameterGrid, GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler

# Stacking
from pancake.Stacker import *

# Data
from sklearn.datasets import load_boston

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

warnings.filterwarnings('ignore')

# Random seed
seed=123


## Data Loading and Pre-processing

In [2]:
# Get data
boston=load_boston()

X = boston['data']
y = boston['target']

In [3]:
feats = boston["feature_names"]
df_boston = pd.DataFrame(X, columns=feats)
df_boston['MEDV'] = y

In [4]:
def quantileClasses(y, percs=[25,50,75]):
    quantiles = np.percentile(y, percs)
    yq = np.zeros_like(y,dtype=int)
    
    # Categorical yq based on quantiles
    yq[(y>quantiles[0]) & (y < quantiles[1])] = 1
    yq[(y>quantiles[1]) & (y < quantiles[2])] = 2
    yq[(y>quantiles[2])] = 3
    
    return yq

In [5]:
yq = quantileClasses(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=yq, test_size=0.25, random_state=seed)

Let's pre-process and use robust scaler to re-scale:

In [6]:
feats_toLog = ['CRIM','DIS','LSTAT']
df_train = pd.DataFrame(X_train, columns=feats)
df_test = pd.DataFrame(X_test, columns=feats)

for f in feats_toLog:
    df_train[f] = np.log10(df_train[f])
    df_test[f] = np.log10(df_test[f])

Let's also rescale the features (except the categorical `CHAS`):

In [7]:
feats_to_normalize = [f for f in feats if f != 'CHAS']
X_ = df_train[feats_to_normalize].values

# Scale training data
scaler = RobustScaler()
X_rscl = scaler.fit_transform(X_)

center_, scale_ = scaler.center_, scaler.scale_

Training and test sets:

In [8]:
# Train 
df_train_new = pd.DataFrame(X_rscl, columns=feats_to_normalize)
df_train_new['CHAS'] = df_train['CHAS']

# Test
X_ = df_test[feats_to_normalize].values
X_ = (X_ - center_) / scale_

df_test_new = pd.DataFrame(X_, columns=feats_to_normalize)
df_test_new['CHAS'] = df_test['CHAS']

## Modeling

In [9]:
X_train = df_train_new[feats].values
X_test = df_test_new[feats].values

### Random Forest

In [10]:
skf = RepeatedKFold(n_repeats=10,n_splits=5,random_state=seed)
hypers = {'max_features':[2,4,6,8]}

regMod_3 = RandomForestRegressor(n_estimators=50,random_state=seed)
grid = GridSearchCV(estimator=regMod_3, param_grid=hypers, cv=skf, scoring='neg_mean_squared_error', 
                    n_jobs=4)
grid.fit(X_train, y_train)

GridSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x1a22ca5278>,
       error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=123, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'max_features': [2, 4, 6, 8]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [11]:
grid.best_params_

{'max_features': 6}

In [12]:
# Train/test predictions
y_pred_tr = grid.predict(X_train)
mse_tr = mean_squared_error(y_train, y_pred_tr)

y_pred_ts = grid.predict(X_test)
mse_ts = mean_squared_error(y_test, y_pred_ts)

In [13]:
# Performance
print("Training RMSE = {:.4f}".format(np.sqrt(mse_tr)))
print("Test RMSE = {:.4f}".format(np.sqrt(mse_ts)))

Training RMSE = 1.3439
Test RMSE = 3.6175


#### SVR

In [14]:
skf = RepeatedKFold(n_repeats=10,n_splits=5,random_state=seed)
hypers = {'C':np.logspace(-4,4,10), 'gamma':np.logspace(-4,4,10)}

regMod_5 = SVR(kernel='rbf')
grid = GridSearchCV(estimator=regMod_5, param_grid=hypers, cv=skf, scoring='neg_mean_squared_error', 
                    n_jobs=4)
grid.fit(X_train, y_train)

GridSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x1a22ce9320>,
       error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'C': array([1.00000e-04, 7.74264e-04, 5.99484e-03, 4.64159e-02, 3.59381e-01,
       2.78256e+00, 2.15443e+01, 1.66810e+02, 1.29155e+03, 1.00000e+04]), 'gamma': array([1.00000e-04, 7.74264e-04, 5.99484e-03, 4.64159e-02, 3.59381e-01,
       2.78256e+00, 2.15443e+01, 1.66810e+02, 1.29155e+03, 1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [15]:
grid.best_params_

{'C': 166.81005372000558, 'gamma': 0.046415888336127774}

In [16]:
# Train/test predictions
y_pred_tr = grid.predict(X_train)
mse_tr = mean_squared_error(y_train, y_pred_tr)

y_pred_ts = grid.predict(X_test)
mse_ts = mean_squared_error(y_test, y_pred_ts)

In [17]:
# Performance
print("Training RMSE = {:.4f}".format(np.sqrt(mse_tr)))
print("Test RMSE = {:.4f}".format(np.sqrt(mse_ts)))

Training RMSE = 1.8473
Test RMSE = 3.7268


## Stacking

In [18]:
# Metric to maximize (negative RMSE)
def nrmse(y,y_pred):
    return -np.sqrt(mean_squared_error(y,y_pred))

# Folds
splt = KFold(n_splits=5,random_state=seed)

# Initiate stacker
stacker = Stacker(X_train, y_train, splitter=splt, evalMetric=nrmse, family="regression")

# Hyper-parameters
hypers_linear = {'alpha':np.logspace(-2,4,200)}
hypers_rf = {'max_features':[2,4,6,8]}
hypers_svr = {'C':np.logspace(-4,4,10), 'gamma':np.logspace(-4,4,10)}

# Add one in-layer model
stacker.addModelIn(RandomForestRegressor(n_estimators=50), trainable=True, 
                   hyperParameters = hypers_rf)
stacker.addModelIn(SVR(kernel='rbf'), trainable=True, hyperParameters = hypers_svr)

# Add one out-layer model
stacker.addModelOut(Ridge(), hypers_linear)

# Train
predsTrain = stacker.stackTrain()

# Test
predsTest = stacker.stackTest(X_test)

In-layer model : 1 trained, Avg CV score across HO folds: -3.3561
In-layer model : 2 trained, Avg CV score across HO folds: -3.1129
Out-layer model : 1 trained,  CV score = -2.7934


In [19]:
# Train/Test set predictions and performance
mse_tr = mean_squared_error(y_train, predsTrain[0])
rmse_tr = np.sqrt(mse_tr)
print("Ridge Regression RMSE (train) = {:.4f}".format(rmse_tr))

mse_ts = mean_squared_error(y_test, predsTest[0])
rmse_ts = np.sqrt(mse_ts)
print("Ridge Regression RMSE (test) = {:.4f}".format(rmse_ts))

Ridge Regression RMSE (train) = 2.8041
Ridge Regression RMSE (test) = 3.5892


In [20]:
stacker.summary()

Stacked Model Summary:
----------------------------------------
2 in-layer models stacked in 1.94e+01 sec
1 out-layer models trained in 1.01e+00 sec
In-layer summary:
----------------------------------------
2 in-Layer models trained/fitted
Out-layer summary:
----------------------------------------
Out-layer model 1: CV score = -2.7934
Best hyper-parameters: {'alpha': 943.7878277775371}


Again, the stacked model performs slightly better..